<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width = 300, align = "center"></a>

<h1 align=center><font size = 5>Assignment: Notebook for Peer Assignment</font></h1>

# Introduction

Using this Python notebook you will:
1. Understand 3 Chicago datasets  
1. Load the 3 datasets into 3 tables in a Db2 database
1. Execute SQL queries to answer assignment questions 

## Understand the datasets 
To complete the assignment problems in this notebook you will be using three datasets that are available on the city of Chicago's Data Portal:
1. <a href="https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2">Socioeconomic Indicators in Chicago</a>
1. <a href="https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t">Chicago Public Schools</a>
1. <a href="https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2">Chicago Crime Data</a>

### 1. Socioeconomic Indicators in Chicago
This dataset contains a selection of six socioeconomic indicators of public health significance and a “hardship index,” for each Chicago community area, for the years 2008 – 2012.

For this assignment you will use a snapshot of this dataset which can be downloaded from:
https://ibm.box.com/shared/static/05c3415cbfbtfnr2fx4atenb2sd361ze.csv

A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:
https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2



### 2. Chicago Public Schools

This dataset shows all school level performance data used to create CPS School Report Cards for the 2011-2012 school year. This dataset is provided by the city of Chicago's Data Portal.

For this assignment you will use a snapshot of this dataset which can be downloaded from:
https://ibm.box.com/shared/static/f9gjvj1gjmxxzycdhplzt01qtz0s7ew7.csv

A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:
https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t




### 3. Chicago Crime Data 

This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present, minus the most recent seven days. 

This dataset is quite large - over 1.5GB in size with over 6.5 million rows. For the purposes of this assignment we will use a much smaller sample of this dataset which can be downloaded from:
https://ibm.box.com/shared/static/svflyugsr9zbqy5bmowgswqemfpm1x7f.csv

A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:
https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2


### Download the datasets
In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the links below to download and save the datasets (.CSV files):
1. __CENSUS_DATA:__ https://ibm.box.com/shared/static/05c3415cbfbtfnr2fx4atenb2sd361ze.csv
1. __CHICAGO_PUBLIC_SCHOOLS__  https://ibm.box.com/shared/static/f9gjvj1gjmxxzycdhplzt01qtz0s7ew7.csv
1. __CHICAGO_CRIME_DATA:__ https://ibm.box.com/shared/static/svflyugsr9zbqy5bmowgswqemfpm1x7f.csv

__NOTE:__ Ensure you have downloaded the datasets using the links above instead of directly from the Chicago Data Portal. The versions linked here are subsets of the original datasets and have some of the column names modified to be more database friendly which will make it easier to complete this assignment.

### Store the datasets in database tables
To analyze the data using SQL, it first needs to be stored in the database.

While it is easier to read the dataset into a Pandas dataframe and then PERSIST it into the database as we saw in Week 3 Lab 3, it results in mapping to default datatypes which may not be optimal for SQL querying. For example a long textual field may map to a CLOB instead of a VARCHAR. 

Therefore, __it is highly recommended to manually load the table using the database console LOAD tool, as indicated in Week 2 Lab 1 Part II__. The only difference with that lab is that in Step 5 of the instructions you will need to click on create "(+) New Table" and specify the name of the table you want to create and then click "Next". 

<img src = "https://ibm.box.com/shared/static/uc4xjh1uxcc78ks1i18v668simioz4es.jpg">

##### Now open the Db2 console, open the LOAD tool, Select / Drag the .CSV file for the first dataset, Next create a New Table, and then follow the steps on-screen instructions to load the data. Name the new tables as folows:
1. __CENSUS_DATA__
1. __CHICAGO_PUBLIC_SCHOOLS__
1. __CHICAGO_CRIME_DATA__

### Connect to the database 
Let us first load the SQL extension and establish a connection with the database

In [1]:
%load_ext sql

In the next cell enter your db2 connection string. Recall you created Service Credentials for your Db2 instance in first lab in Week 3. From the __uri__ field of your Db2 service credentials copy everything after db2:// (except the double quote at the end) and paste it in the cell below after ibm_db_sa://

<img src ="https://ibm.box.com/shared/static/hzhkvdyinpupm2wfx49lkr71q9swbpec.jpg">

In [2]:
# Remember the connection string is of the format:
# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name
# Enter the connection string for your Db2 on Cloud database instance below
%sql ibm_db_sa://fmt51262:s3719g6-0m701jgp@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB

'Connected: fmt51262@BLUDB'

In [3]:
%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES \
      where TABSCHEMA not in ('SYSIBM', 'SYSCAT', 'SYSSTAT', 'SYSIBMADM', 'SYSTOOLS', 'SYSPUBLIC')

 * ibm_db_sa://fmt51262:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


tabschema,tabname,create_time
FMT51262,EXPLAIN_INSTANCE,2020-03-17 20:48:25.987123
FMT51262,EXPLAIN_STATEMENT,2020-03-17 20:48:26.181246
FMT51262,EXPLAIN_ARGUMENT,2020-03-17 20:48:26.585850
FMT51262,EXPLAIN_OBJECT,2020-03-17 20:48:26.839938
FMT51262,EXPLAIN_OPERATOR,2020-03-17 20:48:26.946884
FMT51262,EXPLAIN_PREDICATE,2020-03-17 20:48:27.153764
FMT51262,EXPLAIN_STREAM,2020-03-17 20:48:27.629591
FMT51262,EXPLAIN_DIAGNOSTIC,2020-03-17 20:48:27.861969
FMT51262,EXPLAIN_DIAGNOSTIC_DATA,2020-03-17 20:48:28.087002
FMT51262,OBJECT_METRICS,2020-03-17 20:48:28.310434


## Problems
Now write and execute SQL queries to solve assignment problems

### Problem 1

##### Find the total number of crimes recorded in the CRIME table

In [5]:
# Rows in Crime table
%sql select count(*) from crime_data

 * ibm_db_sa://fmt51262:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


1
533


### Problem 2

##### Retrieve first 10 rows from the CRIME table


In [42]:
%sql select * from crime_data limit 10

 * ibm_db_sa://fmt51262:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


id,case_number,DATE,block,iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area_number,fbicode,x_coordinate,y_coordinate,YEAR,updatedon,latitude,longitude,location
3512276,HK587712,2004-08-28 17:50:56,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,FALSE,FALSE,911,9,14,58,6,1155838,1873050,2004,2018-02-10 15:50:01,41.80744050,-87.70395585,"(41.8074405, -87.703955849)"
3406613,HK456306,2004-06-26 12:40:00,009XX N CENTRAL PARK AVE,820,THEFT,$500 AND UNDER,OTHER,FALSE,FALSE,1112,11,27,23,6,1152206,1906127,2004,2018-02-28 15:56:25,41.89827996,-87.71640551,"(41.898279962, -87.716405505)"
8002131,HT233595,2011-04-04 05:45:00,043XX S WABASH AVE,820,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,FALSE,FALSE,221,2,3,38,6,1177436,1876313,2011,2018-02-10 15:50:01,41.81593313,-87.62464213,"(41.815933131, -87.624642127)"
7903289,HT133522,2010-12-30 16:30:00,083XX S KINGSTON AVE,840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,FALSE,FALSE,423,4,7,46,6,1194622,1850125,2010,2018-02-10 15:50:01,41.74366532,-87.56246276,"(41.743665322, -87.562462756)"
10402076,HZ138551,2016-02-02 19:30:00,033XX W 66TH ST,820,THEFT,$500 AND UNDER,ALLEY,FALSE,FALSE,831,8,15,66,6,1155240,1860661,2016,2018-02-10 15:50:01,41.77345530,-87.70648047,"(41.773455295, -87.706480471)"
7732712,HS540106,2010-09-29 07:59:00,006XX W CHICAGO AVE,810,THEFT,OVER $500,PARKING LOT/GARAGE(NON.RESID.),FALSE,FALSE,1323,12,27,24,6,1171668,1905607,2010,2018-02-10 15:50:01,41.89644677,-87.64493868,"(41.896446772, -87.644938678)"
10769475,HZ534771,2016-11-30 01:15:00,050XX N KEDZIE AVE,810,THEFT,OVER $500,STREET,FALSE,FALSE,1713,17,33,14,6,1154133,1933314,2016,2018-02-10 15:50:01,41.97284491,-87.70860008,"(41.972844913, -87.708600079)"
4494340,HL793243,2005-12-16 16:45:00,005XX E PERSHING RD,860,THEFT,RETAIL THEFT,GROCERY FOOD STORE,TRUE,FALSE,213,2,3,38,6,1180448,1879234,2005,2018-02-28 15:56:25,41.82387989,-87.61350386,"(41.823879885, -87.613503857)"
3778925,HL149610,2005-01-28 17:00:00,100XX S WASHTENAW AVE,810,THEFT,OVER $500,STREET,FALSE,FALSE,2211,22,19,72,6,1160129,1838040,2005,2018-02-28 15:56:25,41.71128051,-87.68917910,"(41.711280513, -87.689179097)"
3324217,HK361551,2004-05-13 14:15:00,033XX W BELMONT AVE,820,THEFT,$500 AND UNDER,SMALL RETAIL STORE,FALSE,FALSE,1733,17,35,21,6,1153590,1921084,2004,2018-02-28 15:56:25,41.93929582,-87.71092344,"(41.939295821, -87.710923442)"


In [36]:
%sql select distinct(NAME), COLTYPE, LENGTH from SYSIBM.SYSCOLUMNS where TBNAME = 'crime_data'

 * ibm_db_sa://fmt51262:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


name,coltype,length


### Problem 3

##### How many crimes involve an arrest?

In [18]:
%sql select count(*) from crime_data \
    where arrest='TRUE'

 * ibm_db_sa://fmt51262:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


1
163


In [21]:
%sql select distinct(NAME), COLTYPE, LENGTH from SYSIBM.SYSCOLUMNS where TBNAME = 'CRIME_DATA'

 * ibm_db_sa://fmt51262:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


name,coltype,length
ARREST,VARCHAR,5
BEAT,SMALLINT,2
BLOCK,VARCHAR,35
CASE_NUMBER,VARCHAR,8
COMMUNITY_AREA_NUMBER,SMALLINT,2
DATE,TIMESTMP,10
DESCRIPTION,VARCHAR,46
DISTRICT,SMALLINT,2
DOMESTIC,VARCHAR,5
FBICODE,VARCHAR,3


### Problem 4

##### Which unique types of crimes have been recorded at GAS STATION locations?


In [41]:
%sql select * from crime_data \
    where location_description = 'GAS STATION'

 * ibm_db_sa://fmt51262:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


id,case_number,DATE,block,iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area_number,fbicode,x_coordinate,y_coordinate,YEAR,updatedon,latitude,longitude,location
7505841,HS308747,2010-05-15 09:15:00,004XX E 95TH ST,890,THEFT,FROM BUILDING,GAS STATION,FALSE,FALSE,633,6,9,49,6,1180667,1842071,2010,2018-02-10 15:50:01,41.72189586,-87.61384102,"(41.721895856, -87.613841024)"
4153041,HL005270,2005-07-12 23:00:00,008XX N CICERO AVE,820,THEFT,$500 AND UNDER,GAS STATION,FALSE,FALSE,1111,11,37,25,6,1144207,1904963,2005,2018-02-10 15:50:01,41.89523990,-87.74581479,"(41.895239896, -87.745814792)"
5844831,HN647052,2007-10-14 04:31:26,062XX S DREXEL AVE,1811,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,GAS STATION,TRUE,FALSE,313,3,20,42,18,1183081,1863726,2007,2018-02-28 15:56:25,41.78126375,-87.60432695,"(41.781263748, -87.60432695)"
3046354,HJ753917,2003-11-12 01:40:00,052XX W ADDISON ST,031A,ROBBERY,ARMED: HANDGUN,GAS STATION,FALSE,FALSE,1634,16,38,15,3,1140892,1923436,2003,2018-02-28 15:56:25,41.94599344,-87.75753456,"(41.945993442, -87.757534561)"
8875058,HV548880,2012-11-05 11:15:00,067XX S STATE ST,330,ROBBERY,AGGRAVATED,GAS STATION,FALSE,FALSE,322,3,6,69,3,1177438,1860407,2012,2018-02-10 15:50:01,41.77228548,-87.62511546,"(41.772285481, -87.625115462)"
5827212,HN637742,2007-10-09 12:20:00,025XX N PULASKI RD,1330,CRIMINAL TRESPASS,TO LAND,GAS STATION,TRUE,FALSE,2524,25,31,22,26,1149288,1916658,2007,2018-02-28 15:56:25,41.92723512,-87.72684967,"(41.927235118, -87.726849666)"


Hint: Which column lists types of crimes e.g. THEFT?

### Problem 5

##### In the CENUS_DATA table list all Community Areas whose names start with the letter ‘B’.

In [51]:
%sql select * from census_data \
    where community_area_name like 'B%'

 * ibm_db_sa://fmt51262:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_number,community_area_name,percent_of_housing_crowded,percent_households_below_poverty,percent_aged_16__unemployed,percent_aged_25__without_high_school_diploma,percent_aged_under_18_or_over_64,per_capita_income,hardship_index
19,Belmont Cragin,10.8,18.7,14.6,37.3,37.3,15461,70
47,Burnside,6.8,33.0,18.6,19.3,42.7,12515,79
58,Brighton Park,14.4,23.6,13.9,45.1,39.3,13089,84
60,Bridgeport,4.5,18.9,13.7,22.2,31.3,22694,43
72,Beverly,0.9,5.1,8.0,3.7,40.5,39523,12


In [13]:
%sql select distinct(NAME), COLTYPE, LENGTH from SYSIBM.SYSCOLUMNS where TBNAME = 'PUBLIC_SCHOOLS'

 * ibm_db_sa://fmt51262:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


name,coltype,length
10th Grade PLAN (2009),VARCHAR,4
10th Grade PLAN (2010),VARCHAR,4
11th Grade Average ACT (2011),VARCHAR,4
9th Grade EXPLORE (2009),VARCHAR,4
9th Grade EXPLORE (2010),VARCHAR,4
AVERAGE_STUDENT_ATTENDANCE,VARCHAR,6
Adequate_Yearly_Progress_Made_,VARCHAR,3
Average_Teacher_Attendance,VARCHAR,6
COLLEGE_ENROLLMENT,SMALLINT,2
COMMUNITY_AREA_NAME,VARCHAR,22


### Problem 6

##### Which schools in Community Areas 10 to 15 are healthy school certified?

In [12]:
%sql select * from public_schools\
    where (COMMUNITY_AREA_NUMBER between 10 and 15)\
    and HEALTHY_SCHOOL_CERTIFIED = 'Yes'

 * ibm_db_sa://fmt51262:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


School_ID,name_of_school,"Elementary, Middle, or High School",Street_Address,City,State,ZIP_Code,Phone_Number,Link,Network_Manager,Collaborative_Name,Adequate_Yearly_Progress_Made_,Track_Schedule,CPS_Performance_Policy_Status,CPS_Performance_Policy_Level,healthy_school_certified,Safety_Icon,safety_score,Family_Involvement_Icon,Family_Involvement_Score,Environment_Icon,Environment_Score,Instruction_Icon,Instruction_Score,Leaders_Icon,Leaders_Score,Teachers_Icon,Teachers_Score,Parent_Engagement_Icon,Parent_Engagement_Score,Parent_Environment_Icon,Parent_Environment_Score,average_student_attendance,Rate_of_Misconducts__per_100_students_,Average_Teacher_Attendance,Individualized_Education_Program_Compliance_Rate,Pk_2_Literacy__,Pk_2_Math__,Gr3_5_Grade_Level_Math__,Gr3_5_Grade_Level_Read__,Gr3_5_Keep_Pace_Read__,Gr3_5_Keep_Pace_Math__,Gr6_8_Grade_Level_Math__,Gr6_8_Grade_Level_Read__,Gr6_8_Keep_Pace_Math_,Gr6_8_Keep_Pace_Read__,Gr_8_Explore_Math__,Gr_8_Explore_Read__,ISAT_Exceeding_Math__,ISAT_Exceeding_Reading__,ISAT_Value_Add_Math,ISAT_Value_Add_Read,ISAT_Value_Add_Color_Math,ISAT_Value_Add_Color_Read,Students_Taking__Algebra__,Students_Passing__Algebra__,9th Grade EXPLORE (2009),9th Grade EXPLORE (2010),10th Grade PLAN (2009),10th Grade PLAN (2010),Net_Change_EXPLORE_and_PLAN,11th Grade Average ACT (2011),Net_Change_PLAN_and_ACT,College_Eligibility__,Graduation_Rate__,College_Enrollment_Rate__,college_enrollment,General_Services_Route,Freshman_on_Track_Rate__,x_coordinate,y_coordinate,Latitude,Longitude,community_area_number,community_area_name,Ward,Police_District,Location
609995,Rufus M Hitch Elementary School,ES,5625 N McVicker Ave,Chicago,IL,60646,(773) 534-1189,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609995.pdf,O'Hare Elementary Network,NORTH-NORTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 1,Yes,Strong,64,Strong,77,Average,55,Average,58,Strong,65,Strong,70,Strong,55,Strong,54,95.10%,3.7,96.60%,98.60%,73.7,27.6,62.2,51.7,50,67.8,56.5,54.8,52,63.3,22.9,37.5,29.5,24.7,-0.1,0.0,Yellow,Yellow,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,542,30,NDA,1134940.195,1937129.571,41.98367756,-87.77908614,10,NORWOOD PARK,45,16,"(41.98367756, -87.77908614)"


### Problem 7

##### What is the average school Safety Score? 

In [16]:
%sql select avg(SAFETY_SCORE) from public_schools

 * ibm_db_sa://fmt51262:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


1
49.504873


### Problem 8

##### List the top 5 Community Areas by average College Enrollment [number of students] 

In [28]:
%sql select community_area_name, COLLEGE_ENROLLMENT from public_schools\
    order by college_enrollment desc limit 5

 * ibm_db_sa://fmt51262:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_name,college_enrollment
NORTH CENTER,4368
ARCHER HEIGHTS,3320
NORWOOD PARK,2922
BRIGHTON PARK,2883
IRVING PARK,2366


### Problem 9

##### Use a sub-query to determine which Community Area has the least value for school Safety Score? 

In [3]:
%sql select community_area_name, safety_score from public_schools\
    where safety_score in (select min(safety_score) from public_schools)

 * ibm_db_sa://fmt51262:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_name,safety_score
WASHINGTON PARK,1


In [12]:
%sql select * from census_data \
    where community_area_name = 'Washington Park'

 * ibm_db_sa://fmt51262:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_number,community_area_name,percent_of_housing_crowded,percent_households_below_poverty,percent_aged_16__unemployed,percent_aged_25__without_high_school_diploma,percent_aged_under_18_or_over_64,per_capita_income,hardship_index
40,Washington Park,5.6,42.1,28.6,25.4,42.8,13785,88


In [7]:
%sql select distinct(NAME), COLTYPE, LENGTH from SYSIBM.SYSCOLUMNS where TBNAME = 'CENSUS_DATA'

 * ibm_db_sa://fmt51262:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


name,coltype,length
COMMUNITY_AREA_NAME,VARCHAR,22
COMMUNITY_AREA_NUMBER,SMALLINT,2
HARDSHIP_INDEX,SMALLINT,2
PERCENT_AGED_16__UNEMPLOYED,DECIMAL,4
PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA,DECIMAL,4
PERCENT_AGED_UNDER_18_OR_OVER_64,DECIMAL,4
PERCENT_HOUSEHOLDS_BELOW_POVERTY,DECIMAL,4
PERCENT_OF_HOUSING_CROWDED,DECIMAL,4
PER_CAPITA_INCOME,INTEGER,4


### Problem 10

##### [Without using an explicit JOIN operator] Find the Per Capita Income of the Community Area which has a school Safety Score of 1.

In [27]:
%sql select PER_CAPITA_INCOME from CENSUS_DATA\
    where COMMUNITY_AREA_NAME in (select COMMUNITY_AREA_NAME from PUBLIC_SCHOOLS where safety_score= '1')

 * ibm_db_sa://fmt51262:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


per_capita_income


Copyright &copy; 2018 [cognitiveclass.ai](cognitiveclass.ai?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).
